# Robot for training 

## Introduction 
For this project, let us create the conpcept of a robot for NLP tasks 
here we are working with a class to controll the overall flow of the program. 
in order to have a more controlled environment to train. 


In [3]:
import pandas as pd
import numpy as np
import string, os
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
# set seeds for reproducability
#from tensorflow import set_random_seed
import tensorflow as tf
tf.random.set_seed(2) 
from numpy.random import seed
#set_random_seed(2)
seed(1)
# keras module for building LSTM 
#from keras.preprocessing.sequence import pad_sequences
from keras_preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku
from keras.utils.data_utils import get_file
import random

from __future__ import print_function
from keras.callbacks import LambdaCallback

2022-11-30 21:13:56.247215: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-30 21:13:56.662172: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-30 21:13:56.759717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-30 21:13:56.759751: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [5]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer
import re
import sys

In [6]:
class robot_predictor_trainable: 
    def __init__(self):
        #here we have the global variables.so far we need 
        #1. the data frames 
        # the model architecture   
        self.df_class_1 = None
        self.df_class_2 = None
        #the tokenizer for each class
        self.tokenizer_class_1 = None
        self.tokenizer_class_2 = None

        #here we need one corpurs per each class
        self.corpus_1 = None
        self.corpus_2 = None
        
        #we need two different tokenizers
        self.tokenizer_1 = Tokenizer()
        self.tokenizer_2 = Tokenizer()
        
        #the models to save
        self.model_1 = None
        self.model_2 = None
        
        #Metadata for trainning class1
        self.total_words_class1=None
        self.predictors_class1=None
        self.label_class1=None
        self.max_sequence_len_class1=None
        
        #Metadata for trainning class2
        self.total_words_class2=None
        self.predictors_class2=None
        self.label_class2=None
        self.max_sequence_len_class2=None
        
        
                
        
    #methods for cleaning the tweets
    #preprocessing methods
    def scrub_words(self,text):
        # remove html markup
        import re
        text=re.sub("(<.*?>)","",text)
    
        #remove non-ascii and digits
        text=re.sub("(\\W|\\d)"," ",text)
  
        #remove whitespace
        text=text.strip()
        return text
        
    #Noise removal, stop word removal, normalizing?
    def cleanString(self,s, special_chars = "\":,.@|ðÿœžðÿâœœïÿœžÿºÿÿœžÿ"):
        from nltk.tokenize import TweetTokenizer
        from nltk.corpus import stopwords



        for char in special_chars:
            s = s.replace(char, "")
        s = s.replace("\n", "")
        s = s.replace("https", "")
        s = self.scrub_words(s)
        tokenizer = TweetTokenizer()
        stop_words = set(stopwords.words('english'))
        cleaned_words = [w for w in tokenizer.tokenize(s) if w not in stop_words]
        return " ".join(cleaned_words)
    
    def cleanFrame(self,frame):
        frame['clean_tweet'] = frame.tweet.apply(self.cleanString)
    

        
        
    
        
    #The methods avaialabe to Load the information  
    def load_dataset_class_1(self):
        df_mexico = pd.read_csv('mexico_en_.csv')
        #lnguage = 'en'
        #df_mexico_1 = df_mexico.loc[df_mexico['lang']==lnguage]        
        t_1=pd.DataFrame({'tweet':list(df_mexico['text'])})
        #load the data to the class
        self.cleanFrame(t_1)
        self.df_class_1 = t_1
        t_1.to_csv('clean_data_c_1.csv')
        print('Data loaded successfully')

    def load_dataset_class_2(self):
        df_argentina = pd.read_csv('argentina_en.csv')
        #lnguage = 'en'
        #df_argentina_1 = df_argentina.loc[df_argentina['lang']==lnguage]        
        t_1=pd.DataFrame({'tweet':list(df_argentina['text'])})
        #load the data to the class
        self.cleanFrame(t_1)
        self.df_class_2 = t_1
        t_1.to_csv('clean_data_c_2.csv')

        print('Data loaded successfully')
        
        
    #=====================================================#
    
    
    #once we have loaded the data we need to generate the corpus per
    #each class, so far the methodology is the next, 
    
    def clean_text(self,txt):
        txt = "".join(t for t in txt if t not in string.punctuation).lower()
        txt = txt.encode("utf8").decode("ascii",'ignore')
        return txt
    
    
    #tokenizer = Tokenizer()
    def get_sequence_of_tokens_1(self,corpus):
        ## tokenization
        self.tokenizer_1.fit_on_texts(corpus)
        total_words = len(self.tokenizer_1.word_index) + 1
    
        ## convert data to a token sequence 
        input_sequences = []
        for line in corpus:
            token_list = self.tokenizer_1.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[:i+1]
                input_sequences.append(n_gram_sequence)
        return input_sequences, total_words
    
    
    
    def get_sequence_of_tokens_2(self,corpus):
        ## tokenization
        self.tokenizer_2.fit_on_texts(corpus)
        total_words = len(self.tokenizer_2.word_index) + 1
    
        ## convert data to a token sequence 
        input_sequences = []
        for line in corpus:
            token_list = self.tokenizer_2.texts_to_sequences([line])[0]
            for i in range(1, len(token_list)):
                n_gram_sequence = token_list[:i+1]
                input_sequences.append(n_gram_sequence)
        return input_sequences, total_words
    
    
    
    
    
    
    
    def generate_padded_sequences(self,input_sequences,total_words):
        max_sequence_len = max([len(x) for x in input_sequences])
        input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    
        predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
        label = ku.to_categorical(label, num_classes=total_words)
        return predictors, label, max_sequence_len
    
    #*******************
    def create_model(self,max_sequence_len,total_words):
        input_len = max_sequence_len - 1
        model = Sequential()
        # ----------Add Input Embedding Layer
        model.add(Embedding(total_words,150, input_length=input_len))
        # ----------Add Hidden Layer 1 - LSTM Layer
        model.add(LSTM(700))
        model.add(Dropout(0.3))
        
        # ----------Add Output Layer
        model.add(Dense(total_words, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer='adam')
        return model
    
    #>>>>>>>>>>>>>>>>>>>>>>>><
    #this method is done to create the corpus and train a model in that corpus 
    
    
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    #self.total_words_class1=None
    #self.predictors_class1=None
    #self.label_class1=None
    #self.max_sequence_len_class1=None
    
    def create_corpus_clas_1(self):
        #create the corpus for self.df_class_1
        all_headlines = list(self.df_class_1['clean_tweet'])
        corpus = [self.clean_text(x) for x in all_headlines]
        self.corpus_1=corpus
        print(corpus[:10])
        #token created at the beginning
        #tokenizer = Tokenizer()
        #tokenize the corpus
        inp_sequences,self.total_words_class1 = self.get_sequence_of_tokens_1(corpus)
        print(inp_sequences[:10])
        
        #generate padding sequences 
        self.predictors_class1,self.label_class1,self.max_sequence_len_class1= self.generate_padded_sequences(inp_sequences,self.total_words_class1)
        
      
        

        
    #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
    
       
    def create_corpus_clas_2(self):
        #create the corpus for self.df_class_1
        all_headlines = list(self.df_class_2['clean_tweet'])
        corpus = [self.clean_text(x) for x in all_headlines]
        self.corpus_2=corpus
        print(corpus[:10])
        #token created at the beginning
        #tokenizer = Tokenizer()
        #tokenize the corpus
        inp_sequences,self.total_words_class2 = self.get_sequence_of_tokens_2(corpus)
        print(inp_sequences[:10])
        
        #generate padding sequences 
        self.predictors_class2,self.label_class2,self.max_sequence_len_class2= self.generate_padded_sequences(inp_sequences,self.total_words_class2)
        
      
        
    ##################################################
    #Training 
    def train_class_1(self):
        #use to train 
        model = self.create_model(self.max_sequence_len_class1,self.total_words_class1)
        model.summary()
        
        model.fit(self.predictors_class1,self.label_class1, epochs=20, verbose=5)
        self.model_1 = model
        model.save('Training_class_1.h5')
        
        #Training 
    def train_class_2(self):
        #use to train 
        model = self.create_model(self.max_sequence_len_class2,self.total_words_class2)
        model.summary()
        #print_callback = LambdaCallback(on_epoch_end=self.on_epoch_end)
        model.fit(self.predictors_class2,self.label_class2, epochs=20, verbose=5)
        self.model_2 = model
        model.save('Training_class_2.h5')
        
    
    
    #############################################
    #call back functions
    def sample(self,preds, temperature=1.0):
        # helper function to sample an index from a probability array
        preds = np.asarray(preds).astype('float64')
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        probas = np.random.multinomial(1, preds, 1)
        return np.argmax(probas)


    def on_epoch_end(self,epoch, _):
        # Function invoked at end of each epoch. Prints generated text.
        
        
        
        
        
        print()
        print('----- Generating text after Epoch: %d' % epoch)
        text=str(self.corpus_2)
        
        chars = sorted(list(set(text)))
        print('total chars:', len(chars))
        char_indices = dict((c, i) for i, c in enumerate(chars))
        indices_char = dict((i, c) for i, c in enumerate(chars))

        # cut the text in semi-redundant sequences of maxlen characters
        maxlen = 40
        step = 3
        sentences = []
        next_chars = []
        for i in range(0, len(text) - maxlen, step):
            sentences.append(text[i: i + maxlen])
            next_chars.append(text[i + maxlen])
        print('nb sequences:', len(sentences))

        #print('Vectorization...')
        #x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
        #y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
        #for i, sentence in enumerate(sentences):
         #   for t, char in enumerate(sentence):
          #      x[i, t, char_indices[char]] = 1
           # y[i, char_indices[next_chars[i]]] = 1

        
        
        
        
        maxlen=self.max_sequence_len_class2
        start_index = random.randint(0,len(text) - maxlen - 1)
        for diversity in [0.2, 0.5, 1.0, 1.2]:
            print('----- diversity:', diversity)

            generated = ''
            sentence = text[start_index: start_index + maxlen]
            generated += sentence
            print('----- Generating with seed: "' + str(sentence) + '"')
            sys.stdout.write(generated)

            for i in range(400):
                x_pred = np.zeros((1, maxlen, len(chars)))
                for t, char in enumerate(sentence):
                    x_pred[0, t, char_indices[char]] = 1.

                preds = model.predict(x_pred, verbose=0)[0]
                next_index = sample(preds, diversity)
                next_char = indices_char[next_index]

                sentence = sentence[1:] + next_char

                sys.stdout.write(next_char)
                sys.stdout.flush()
            print()


    
    
    
    
    
    
    ###############################################
    #self.total_words_class1=None
    #self.predictors_class1=None
    #self.label_class1=None
    #self.max_sequence_len_class1=None
    #methods to generate Text 
    #here we need to create one tweet of a given class to the other class.
    
    # 1. FROM 1 to 2 
    # 2. FROM 2 to 1
    
    
    def generate_text_from_1_to_1(self,seed_text, next_words):
        #the main workflow is twett_class_1 -> encode_class1-> z -> decode_class_2
        model = tf.keras.models.load_model('Training_class_1.h5')
        gen_w = ""
        
        #enconding_class1 
        for _ in range(next_words):
            token_list = self.tokenizer_1.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences([token_list],maxlen=self.max_sequence_len_class1-1, padding='pre')
            #predicted = model.predict_classes(token_list, verbose=0)
            #decode with model_class_2
            predicted = np.argmax(model.predict(token_list), axis=-1)
            output_word = ""
            #descompress for argentina xd
            for word,index in self.tokenizer_1.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " "+output_word
            gen_w += " "+output_word
            
        return gen_w.title()
    
    

    def generate_text_from_2_to_2(self,seed_text, next_words):
        #the main workflow is twett_class_1 -> encode_class1-> z -> decode_class_2
        model = tf.keras.models.load_model('Training_class_2.h5')
        gen_w = ""
        
        #enconding_class1 
        for _ in range(next_words):
            token_list = self.tokenizer_2.texts_to_sequences([seed_text])[0]
            token_list = pad_sequences([token_list],maxlen=self.max_sequence_len_class2-1, padding='pre')
            #predicted = model.predict_classes(token_list, verbose=0)
            #decode with model_class_2
            predicted = np.argmax(model.predict(token_list), axis=-1)
            output_word = ""
            #descompress for argentina xd
            for word,index in self.tokenizer_2.word_index.items():
                if index == predicted:
                    output_word = word
                    break
            seed_text += " "+output_word
            gen_w += " "+output_word
            
        return gen_w.title()

    

        

        
        

        
        
        
    
    
    
    

In [7]:
#Start the class
robot = robot_predictor_trainable()

In [8]:
#load the data 

robot.load_dataset_class_1()

robot.load_dataset_class_2()


Data loaded successfully
Data loaded successfully


In [9]:
#create copus class 1
robot.create_corpus_clas_1()

['oh wow ricardo la volpe mauro camoranesi starred fun moment discussed importance seleccionmexicana reaching fifth game qatar they also sent harsh message liga mx tco gmzkdhebby', 'many fans seleccinmexicana selling tickets mexico vs saudiarabia game allaboutqatar tco icrv e oty', 'meanwhile little princesses misnamed seleccionmexicana mind photograph relatives players score goalsbut hey expected fans still receive returning worldcupmiseleccionmx tco j gkjhz x', 'send us one seleccionmexicana one femexfutac teams give one forwards tco oc anzrb', 'it seen coming didn notice tata came vacation like eriksson difference eriksson sent hell months tata living almost years seleccionmexicana mexico qatar mundial', 'canelolvarez warns messi social media kicking mexicanselection shirt sports national international canelo qatar tco ytwdityh tco itbyauw', 'after threatening messi sergio kun agero cesc fabregas respond canelo lvarez tco nppixxwyej info info deportes canelo messi copamundialfifa mu

In [10]:
robot.create_corpus_clas_2()

['all calves maria becerra releases album la nena de argentina tco vpwebz', 'lauradimarco taking period argentina conquered high salary us sthe default pays lie another q macriesmufa', 'bluish edutorresr antonionelli nono hippocrita would disgusted goals argentina celebrated hypothetical goals mexicoclosing comment volpe moderately known argentine coach really feels mexican flag argentina', 'edufek joke bad taste country kingdom upside argentine case beautiful country people work', 'eldi brunodigennaro alvaritomorales football game win argentina involve issues do need boys hug', 'kaeldelaj brunoformiga so switzerland better france saudi arabia team argentina copa america', 'the problem goes result belgium argentina copa america tco odhat zrg', 'ndyws yes yes take cat matches argentina mufeti family', 'the economist also defined plan argentina move forward negotiable implies reform state lowering public spending adjustment instead falling people falls political caste', 'bayernhost i rea

In [ ]:
#training 
robot.train_class_1()

2022-11-30 21:14:10.218189: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-30 21:14:10.218452: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-30 21:14:10.218473: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (augusto-Aspire-A515-55): /proc/driver/nvidia/version does not exist
2022-11-30 21:14:10.219367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 150)           1669350   
                                                                 
 lstm (LSTM)                 (None, 700)               2382800   
                                                                 
 dropout (Dropout)           (None, 700)               0         
                                                                 
 dense (Dense)               (None, 11129)             7801429   
                                                                 
Total params: 11,853,579
Trainable params: 11,853,579
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20


In [71]:
robot.train_class_2()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 26, 300)           80400     
                                                                 
 lstm_10 (LSTM)              (None, 1500)              10806000  
                                                                 
 dropout_10 (Dropout)        (None, 1500)              0         
                                                                 
 dense_10 (Dense)            (None, 268)               402268    
                                                                 
Total params: 11,288,668
Trainable params: 11,288,668
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
Epoch 2/20
Epoch 3/20
Epoch 4/20
Epoch 5/20
Epoch 6/20
Epoch 7/20
Epoch 8/20
Epoch 9/20
Epoch 10/20
Epoch 11/20
Epoch 12/20
Epoch 13/20
Epoch 14/20
Epoch 15/20
Ep

# Generate Text

In [77]:
df=pd.read_csv('clean_data_c_1.csv')
#df.head()
c_1 = list(df['clean_tweet'])
df


,Unnamed: 0,tweet,clean_tweet
0,0,📊Numbers of the #SeleccionMexicana 🇲🇽 in World...,Numbers SeleccionMexicana World Cups Won Ties ...
1,1,The joy of seeing your hero perform at the #FI...,The joy seeing hero perform FIFAWorldCup fifaw...
2,2,The joy of seeing your hero perform at the #FI...,The joy seeing hero perform FIFAWorldCup fifaw...
3,3,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,PAY US TO HELP IN YOUR DUEHomeworkAssignmentsE...
4,4,PAY US TO HELP IN YOUR DUE\n\nHomework\nAssign...,PAY US TO HELP IN YOUR DUEHomeworkAssignmentsE...
...,...,...,...
143,143,"📌 @equipoayb\n📅 Saturday November 2nd 6th, 202...",equipoayb Saturday November nd th fifaworldcup...
144,144,"📌 @equipoayb\n📅 Saturday November 2nd 6th, 202...",equipoayb Saturday November nd th fifaworldcup...
145,145,"📌 @equipoayb\n📅 Saturday November 2nd 6th, 202...",equipoayb Saturday November nd th fifaworldcup...
146,146,@ALEXPAEZ29 @SirJohnLaguna @miseleccionmx True,ALEXPAEZ SirJohnLaguna miseleccionmx True


In [78]:
df2=pd.read_csv('clean_data_c_2.csv')
#df.head()
c_2 = list(df2['clean_tweet'])
df2

,Unnamed: 0,tweet,clean_tweet
0,0,@LigaAFA_Eng @bff_football @JohnCena Unbelieva...,LigaAFA_Eng bff_football JohnCena Unbelievable...
1,1,@budweiser_ar Unbelievable crowd to watch Arge...,budweiser_ar Unbelievable crowd watch Argentin...
2,2,@golazoargentino Unbelievable crowd to watch A...,golazoargentino Unbelievable crowd watch Argen...
3,3,@sudanalytics_ Unbelievable crowd to watch Arg...,sudanalytics _ Unbelievable crowd watch Argent...
4,4,We are not going to give up.🤙\nWe are Argentin...,We going give We Argentina Selecciónmayor Albi...
5,5,@dataref_ar Unbelievable crowd to watch Argent...,dataref_ar Unbelievable crowd watch Argentina ...
6,6,@DiarioOle @SobugMessi10jr Unbelievable crowd ...,DiarioOle SobugMessi jr Unbelievable crowd wat...
7,7,@ESPNFC @SobugMessi10jr Unbelievable crowd to ...,ESPNFC SobugMessi jr Unbelievable crowd watch ...
8,8,@sudanalytics_ Unbelievable crowd to watch Arg...,sudanalytics _ Unbelievable crowd watch Argent...
9,9,We are not going to give up.🤙\nWe are Argentin...,We going give We Argentina Selecciónmayor Albi...


In [80]:
c_1[56]

'One last dance SiSePuede tco GgVPMf zL'

In [81]:
robot.generate_text_from_2_to_2(seed_text=c_1[56],next_words=60)

1/1 [==============================] - 0s 111ms/step


' Sobugmessi Jr Unbelievable Crowd Watch Argentina Mexico Match Dhaka Universitybangladesh Seleccinmayor Albicelestaforever Vamosargentina Mexicovsargentina Messi Worldcup Argentina Goat Bangladesh Worldcupqatar Qatar Tco Eim I K Eu Eu Eu Eu Eu Eu Zp Zp Zp Zp Zp Zp Zp Zp Zp Wx Wx Definition Nowisall Nowisall Nowisall Nowisall Tco Wx Wx Nowisall Worldcupqatar Worldcupqatar Worldcupqatar Tco Tj Cqkjkbx Fky Ee Mundialqatar'

In [17]:
robot.generate_text_from_1_to_1(seed_text=c_2[0],next_words=140)

1/1 [==============================] - 0s 127ms/step


' Team Team Team Win Miseleccionmxen Fifaworldcup Fifaworldcup Seleccionmexicana Seleccionmexicana Seleccionmexicana Argentina Mqj Mqj Polska Polska Saudiarabia Poland Tco Qwvypilkut Tco Qwvypilkut Tco Bzzeyrk N N N N N Ivq Ivq Ivq Ivq H H H H H H H Relying Football Veterans Veterans Need We Keep Athletes Athletes Mexico Mexico Football Qatar Seleccionmexicana Qatar Qatar Worldcup Mexicodemivida Mexicodemivida Tco Fspav Poland Tco Qwvypilkut Qwvypilkut Qwvypilkut Tco Qwvypilkut Tco N H H H H Ivq Ivq Ivq H H H H H H H H Tos Difference Seleccionmexicana Qatar Qatar Qatar Worldcup Afa Mexicodemivida Polska Ch Poland Tco Qwvypilkut Ivq Ivq Ivq Tco Ivq Ivq Ivq Ivq Ivq Ivq Ivq H H H H H H H H H H Npz Npz H Difference Difference Difference Argentina Athletes Mexico Seleccionmexicana Qatar Qatar Tco Fspav Tco Fspav Ivq Ivq Ivq Ivq Ivq'